In [5]:
import earthaccess
earthaccess.login(strategy="netrc")
print("Authenticated")

Authenticated


In [15]:
SHORT_NAME = "SWOT_L4_HR_DAWG_SOS_DISCHARGE_V3"

granules = earthaccess.search_data(
    short_name=SHORT_NAME,
    sort_key="-start_date",
    count=1
)

print("Granules found:", len(granules))

granules = earthaccess.search_data(
    short_name=SHORT_NAME,
    count=50  # enough to list all regions
)

eu_granules = [
    g for g in granules
    if g["meta"]["native-id"].startswith("eu_")
]

paths = earthaccess.download(
    eu_granules,
    local_path="./downloaded_files/sos_europe"
)

paths

print(paths[0])

 

Granules found: 1


QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<?, ?it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [36:57<00:00, 2217.52s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<?, ?it/s]

downloaded_files\sos_europe\eu_sword_v16_SOS_results_unconstrained_20230502T204408_20250502T204408_20251219T163700.nc


In [19]:
import os

NC_PATH = r"downloaded_files\sos_europe\eu_sword_v16_SOS_results_unconstrained_20230502T204408_20250502T204408_20251219T163700.nc"
print("Exists:", os.path.exists(NC_PATH))
print("Size (MB):", os.path.getsize(NC_PATH)/1e6)


Exists: True
Size (MB): 3347.054592


In [20]:
with open(NC_PATH, "rb") as f:
    header = f.read(8)
print(header)


b'\x89HDF\r\n\x1a\n'


In [22]:
import h5py
with h5py.File(NC_PATH, "r") as f:
    print(list(f.keys()))


OSError: Unable to open file (truncated file: eof = 3347054592, sblock->base_addr = 0, stored_eof = 7421350773)

In [24]:
import os
import netCDF4 as nc
import sys

print("Python:", sys.version)
print("netCDF4:", nc.__version__)
print("NC_PATH exists:", os.path.exists(NC_PATH))
print("NC_PATH size (MB):", os.path.getsize(NC_PATH)/1e6)

try:
    ds = nc.Dataset(NC_PATH, "r")
    print("OPENED ✅")
    print("Groups:", list(ds.groups.keys()))
    ds.close()
except Exception as e:
    print("FAILED ❌")
    print(type(e).__name__ + ":", e)


Python: 3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)]
netCDF4: 1.7.2
NC_PATH exists: True
NC_PATH size (MB): 3347.054592
FAILED ❌
OSError: [Errno -101] NetCDF: HDF error: 'downloaded_files\\sos_europe\\eu_sword_v16_SOS_results_unconstrained_20230502T204408_20250502T204408_20251219T163700.nc'


In [23]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np

NC_PATH = r"downloaded_files\sos_europe\eu_sword_v16_SOS_results_unconstrained_20230502T204408_20250502T204408_20251219T163700.nc"
REACH_ID = 22511300021  # must be one of your valid consensus reaches

ds = nc.Dataset(NC_PATH, "r")

reaches = ds.groups["reaches"]
consensus = ds.groups["consensus"]

# --- find reach index ---
reach_ids = reaches.variables["reach_id"][:].astype("int64")
idx = np.where(reach_ids == REACH_ID)[0]
if idx.size == 0:
    ds.close()
    raise ValueError(f"Reach {REACH_ID} not found in file.")

i = int(idx[0])

# --- time for this reach (vlen) ---
time_var = consensus.variables["time_int"]
times = np.asarray(time_var[i], dtype="float64")

valid_time = times > -9.0e10
times_valid = times[valid_time].astype("int64")

datetimes = np.array([
    np.datetime64("2000-01-01") + np.timedelta64(int(t), "s")
    for t in times_valid
])

# --- consensus discharge for this reach (vlen) ---
qvar = consensus.variables["consensus_q"]
q = np.asarray(qvar[i], dtype="float64")[valid_time]

# missing handling
missing = None
if "_FillValue" in qvar.ncattrs():
    missing = qvar.getncattr("_FillValue")
elif "missing_value" in qvar.ncattrs():
    missing = qvar.getncattr("missing_value")

if missing is not None:
    q[q == missing] = np.nan

q[q <= -9.0e10] = np.nan

# --- plot ---
plt.figure(figsize=(10, 5))
plt.plot(datetimes, q, marker="o")
plt.title(f"Consensus discharge for reach {REACH_ID}")
plt.xlabel("Time")
plt.ylabel("Discharge (m³/s)")
plt.grid(True)
plt.tight_layout()
plt.show()

ds.close()


OSError: [Errno -101] NetCDF: HDF error: 'downloaded_files\\sos_europe\\eu_sword_v16_SOS_results_unconstrained_20230502T204408_20250502T204408_20251219T163700.nc'